In [2]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
import numpy as np
import pickle

np.random.seed(1234)

URM_all = getURM_all()

with open('URM_train.pkl', 'rb') as f:
    URM_train = pickle.load(f)
with open('URM_validation.pkl', 'rb') as f:
    URM_validation = pickle.load(f)
    
with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)
with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [3]:
# SETUP EVALUATORS
from Evaluation.Evaluator import EvaluatorHoldout

#evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions


In [5]:
from Hybrid import TwoScoresHybridRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
import pickle


MAP_recommender_per_group = {}

collaborative_recommender_class = {
                                   "UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "SLIM_ELASTIC": SLIMElasticNetRecommender,
                                    "MULT_VAE": MultVAERecommender_OptimizerMask,
                                    "IALS": IALSRecommender
                                   }
collaborative_recommender_class.items()

recommender_object_dict_train = {}
recommender_object_dict_train_validation = {}
recommender_object_dict_all = {}
for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train)
    recommender_object_dict_train[label] = recommender_object
    
    recommender_object = recommender_class(URM_train_validation)
    recommender_object_dict_train_validation[label] = recommender_object
    
    recommender_object = recommender_class(URM_all)
    recommender_object_dict_all[label] = recommender_object
    
    
# recommender_object_dict_train["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
# recommender_object_dict_train["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
# recommender_object_dict_train["RP3beta"].fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
# recommender_object_dict_train["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
#                                          feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
# recommender_object_dict_train["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
#                                        feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
# recommender_object_dict_train["TOP_POP"].fit()
# ials = IALSRecommender(URM_train)
# ials.load_model("slim_models/", "IALS_train")
# recommender_object_dict_train["IALS"] = ials
# 
# mv = MultVAERecommender_OptimizerMask(URM_train)
# mv.load_model("slim_models/", "MultVAE_train_300")
# recommender_object_dict_train["MULT_VAE"] = mv

recommender_object_dict_train_validation["SLIM_ELASTIC"].load_model("slim_models/", "slim_t_val")
recommender_object_dict_train_validation["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict_train_validation["RP3beta"].fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
recommender_object_dict_train_validation["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict_train_validation["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                        feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
final = MultVAERecommender_OptimizerMask(URM_train_validation)
final.load_model("slim_models/", "MultVAE_train_validation_300")
mv = final
recommender_object_dict_train_validation["MULT_VAE"] = mv
ials = IALSRecommender(URM_train)
ials.load_model("slim_models/", "IALS_train_validation")
recommender_object_dict_train_validation["IALS"] = ials

recommender_object_dict_train_validation["hybrid2"] = TwoScoresHybridRecommender(URM_train_validation, recommender_object_dict_train_validation["SLIM_ELASTIC"], recommender_object_dict_train_validation["RP3beta"])
recommender_object_dict_train_validation["hybrid2"].fit(alpha=0.6201320790279279)

# recommender_object_dict_train_validation["MULT_VAE"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)

recommender_object_dict_all["SLIM_ELASTIC"].load_model("slim_models/", "slim_24")
recommender_object_dict_all["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict_all["RP3beta"].fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
recommender_object_dict_all["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict_all["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                        feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
final = MultVAERecommender_OptimizerMask(URM_all)
final.load_model("slim_models/", "MultVAE_all_opt_300")
mv = final
recommender_object_dict_all["MULT_VAE"] = mv
ials = IALSRecommender(URM_train)
ials.load_model("slim_models/", "IALS_all")
recommender_object_dict_all["IALS"] = ials

recommender_object_dict_all["hybrid2"] = TwoScoresHybridRecommender(URM_all, recommender_object_dict_all["SLIM_ELASTIC"], recommender_object_dict_all["RP3beta"])
recommender_object_dict_all["hybrid2"].fit(alpha=0.6201320790279279)


UserKNNCFRecommender: URM Detected 422 ( 3.3%) users with no interactions.
UserKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
UserKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 422 ( 3.3%) users with no interactions.
ItemKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
P3alphaRecommender: URM Detected 422 ( 3.3%) users with no interactions.
P3alphaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
P3alphaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: URM Detected 422 ( 3.3%) users with no interactions.
RP3betaRecommender: URM Detected 32

2024-01-07 00:22:24.869966: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 00:22:24.870340: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 00:22:24.870593: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Restoring parameters from slim_models/MultVAE_train_validation_300/.session/session
MultVAERecommender: Loading complete
IALSRecommender: URM Detected 422 ( 3.3%) users with no interactions.
IALSRecommender: URM Detected 329 ( 1.5%) items with no interactions.
IALSRecommender: Loading model from file 'slim_models/IALS_train_validation'
IALSRecommender: Loading complete
SLIMElasticNetRP3betaTwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'slim_models/slim_24'
SLIMElasticNetRecommender: Loading complete
P3alphaRecommender: Similarity column 22222 (100.0%), 3447.94 column/sec. Elapsed time 6.45 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 3390.50 column/sec. Elapsed time 6.55 sec
Similarity column 12638 (100.0%), 4395.72 column/sec. Elapsed time 2.88 sec
Similarity column 22222

2024-01-07 00:22:52.499169: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 00:22:52.499585: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 00:22:52.499931: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Restoring parameters from slim_models/MultVAE_all_opt_300/.session/session
MultVAERecommender: Loading complete
IALSRecommender: URM Detected 422 ( 3.3%) users with no interactions.
IALSRecommender: URM Detected 329 ( 1.5%) items with no interactions.
IALSRecommender: Loading model from file 'slim_models/IALS_all'
IALSRecommender: Loading complete


In [11]:
from xgboost import XGBRanker
import pandas as pd

from Recommenders.BaseRecommender import BaseRecommender
user_id_to_index_mapper = get_user_id_to_index_mapper()
data_target = pd.read_csv(filepath_or_buffer="data_target_users_test.csv",
                                               sep=",",
                                               # header=None,
                                               dtype={0: int},
                                               engine='python')
data_target["user_id"] = data_target["user_id"].map(user_id_to_index_mapper)
data_target = data_target.dropna()
data_target["user_id"] = data_target["user_id"].astype(int)


class XGBoostRecommender(BaseRecommender):

    RECOMMENDER_NAME = "XGBoostRecommender"

    def __init__(self,
                URM_train,
                recommenders,
                recommenders_2,
                main_recommender,
                verbose = True,
                n_estimators = 50,
                learning_rate = 1e-1,
                reg_alpha = 1e-1,
                reg_lambda = 1e-1,
                max_depth = 20,
                max_leaves = 0,
                grow_policy = "depthwise",
                objective = "pairwise",
                booster = "gbtree",
                random_seed = None,
                ):

        super(XGBoostRecommender, self).__init__(URM_train, verbose=verbose)

        self.XGB_model = XGBRanker(objective='rank:{}'.format(objective),
                                n_estimators = int(n_estimators),
                                random_state = random_seed,
                                learning_rate = learning_rate,
                                reg_alpha = reg_alpha,
                                reg_lambda = reg_lambda,
                                max_depth = int(max_depth),
                                max_leaves = int(max_leaves),
                                grow_policy = grow_policy,
                                verbosity = 0, # 2 if self.verbose else 0,
                                booster = booster,
                                enable_categorical = True,
                            )
        self.recommenders = recommenders
        self.recommenders_2 = recommenders_2
        self.main_recommender = main_recommender
        self.X_train = None
        self.y_train = None
        
    def fit(self):
        xgboost_values = []
        xgboost_values_for_item_score = []
        suggestions, scores_MAIN = self.recommenders[self.main_recommender].recommend(data_target["user_id"].to_numpy(), cutoff=30, return_scores=True)
        suggestions_for_item_score, _ = self.recommenders_2[self.main_recommender].recommend(data_target["user_id"].to_numpy(), cutoff=30, return_scores=True)

        other_list = [recommender for recommender in self.recommenders if recommender != self.main_recommender]
        other_list_for_item_score = [recommender for recommender in self.recommenders_2 if recommender != self.main_recommender]
        
        scores_others = {}
        scores_others_for_item_score = {}
        for name, recommender in self.recommenders.items():
            if name != self.main_recommender:
                _, scores_others[name] = recommender.recommend(data_target["user_id"].to_numpy(), cutoff=30, return_scores=True)
                _, scores_others_for_item_score[name] = self.recommenders_2[name].recommend(data_target["user_id"].to_numpy(), cutoff=30, return_scores=True)
    
        for i, suggestion in enumerate(suggestions):
            for item_id in suggestion:
                label = self.URM_train[data_target["user_id"].to_numpy()[i],item_id] == 1
                item_popularity = self.URM_train[:,item_id].sum()
                user_popularity = self.URM_train[data_target["user_id"].to_numpy()[i],:].sum()
                xgboost_values.append([data_target["user_id"].to_numpy()[i], item_id, label, item_popularity, user_popularity,scores_MAIN[i][item_id]]+[scores_others[other][i][item_id] for other in other_list])
                
                xgboost_values_for_item_score.append([data_target["user_id"].to_numpy()[i], item_id, label, item_popularity, user_popularity,scores_MAIN[i][item_id]]+[scores_others_for_item_score[other][i][item_id] for other in other_list_for_item_score])
        xgboost_df = pd.DataFrame(xgboost_values,columns=["user_id", "item_id", "label","item_popularity", "user_popularity", self.main_recommender]+other_list)
        xgboost_df_for_item_score = pd.DataFrame(xgboost_values_for_item_score,columns=["user_id", "item_id", "label","item_popularity", "user_popularity", self.main_recommender]+other_list_for_item_score)
        
        self.y_train = xgboost_df["label"]
        self.X_train = xgboost_df.drop(columns=["label"])
        self.X_train["user_id"] = self.X_train["user_id"].astype("category")
        self.X_train["item_id"] = self.X_train["item_id"].astype("category")
        
        self.X_train_for_item_score = xgboost_df_for_item_score.drop(columns=["label"])
        self.X_train_for_item_score["user_id"] = self.X_train_for_item_score["user_id"].astype("category")
        self.X_train_for_item_score["item_id"] = self.X_train_for_item_score["item_id"].astype("category")
        
        groups = xgboost_df.groupby("user_id").size().values
        self.XGB_model.fit(self.X_train,
                    self.y_train,
                    group=groups,
                    verbose=True)
    
    # def fit(self):
    #     xgboost_values = []
    #     suggestions, scores_MAIN = self.recommenders[self.main_recommender].recommend(data_target["user_id"].to_numpy(), cutoff=30, return_scores=True)
    #     other_list = [recommender for recommender in self.recommenders if recommender != self.main_recommender]
    #     scores_others = {}
    #     for name, recommender in self.recommenders.items():
    #         if name != self.main_recommender:
    #             _, scores_others[name] = recommender.recommend(data_target["user_id"].to_numpy(), cutoff=30, return_scores=True)
    #     for i, suggestion in enumerate(suggestions):
    #         for item_id in suggestion:
    #             label = self.URM_train[data_target["user_id"].to_numpy()[i],item_id] == 1
    #             item_popularity = self.URM_train[:,item_id].sum()
    #             user_popularity = self.URM_train[data_target["user_id"].to_numpy()[i],:].sum()
    #             xgboost_values.append([data_target["user_id"].to_numpy()[i], item_id, label, item_popularity, user_popularity,scores_MAIN[i][item_id]]+[scores_others[other][i][item_id] for other in other_list])
    #     
    #     # add top pop items to users with 0 label
    #     # toppop_items, scores_toppop = self.recommenders["TopPop"].recommend(1, cutoff=30, return_scores=True)
    #     # for user in data_target["user_id"]:
    #     #     xgboost_values.append([user, toppop_items+1, 0, 0]+[0 for other in other_list if other != "TopPop"]+[scores_toppop[0][item_map[item]] for item in toppop_items])
    #     
    #     xgboost_df = pd.DataFrame(xgboost_values,columns=["user_id", "item_id", "label","item_popularity", "user_popularity", self.main_recommender]+other_list)
    # 
    #     self.y_train = xgboost_df["label"]
    #     self.X_train = xgboost_df.drop(columns=["label"])
    #     self.X_train["user_id"] = self.X_train["user_id"].astype("category")
    #     self.X_train["item_id"] = self.X_train["item_id"].astype("category")
    #     groups = xgboost_df.groupby("user_id").size().values
    #     self.XGB_model.fit(self.X_train,
    #                 self.y_train,
    #                 group=groups,
    #                 verbose=True)
    
    def get_URM_train(self):
        return self.URM_train

    def _compute_item_score(self, user_id_array, items_to_compute):
        item_weights = np.zeros((len(user_id_array), self.n_items))
        for i, user in enumerate(user_id_array):
            if user in data_target["user_id"].values-1:
                X_to_predict = self.X_train_for_item_score[self.X_train_for_item_score["user_id"] == user]
                scores = self.XGB_model.predict(X_to_predict)
                scores = (scores - scores.min())/(scores.max() - scores.min() + 1e-12)
                item_weights[i][X_to_predict["item_id"]] = scores

        return item_weights

In [13]:
main_recommender = "SLIM_ELASTIC"
xg_boost_recommender = XGBoostRecommender(URM_all, recommender_object_dict_train_validation, recommender_object_dict_all, main_recommender)

In [ ]:
xg_boost_recommender.fit()

In [ ]:
xg_boost_recommender.X_train[xg_boost_recommender.X_train["user_id"] == 555]

In [ ]:
from xgboost import plot_importance
plot_importance(xg_boost_recommender.XGB_model)

In [96]:
generateSubmission(xg_boost_recommender)

ValueError: zero-size array to reduction operation minimum which has no identity

In [401]:
# evaluation_res[0]

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.168823,0.259664,0.207566,0.166033,0.256228,0.62453,0.338491,0.1862,0.626884,0.98549,...,0.804822,0.50453,0.804822,0.044584,9.82538,0.99719,0.127975,0.75688,1.81619,0.301498
